# Proyecto Naive Bayes Algorithm

In [1]:
# Librería para la declaración y uso de Data Frames:
import pandas as pd

# Librería para poder realizar la partición del conjunto de datos:
from sklearn.model_selection import train_test_split

# Librería para poder utilizar un vectorizador de textos:
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB
from sklearn.metrics import classification_report

import numpy as np

## Paso 1 - Lectura de Datos:

En primer lugar, es necesario **leer y guardar la información** en una variable para poder empezar a trabajar con ella.

Para ello, se ha guardado el archivo con todos los datos en la ruta: */workspaces/naive-bayes-clara-ab/data/raw/playstore_reviews.csv* y se ha cargado en un Data Frame:

In [2]:
# Lectura del CSV con los datos, dada la ruta donde se guarda el archivo:
df = pd.read_csv ('/workspaces/naive-bayes-clara-ab/data/raw/playstore_reviews.csv');

# Configuración de pandas para mostrar todas las columnas del DataFrame sin truncarlas al visualizarlo
pd.set_option('display.max_columns', None);

# Se muestran las 5 primeras filas del Data Frame
df.head()

,package_name,review,polarity
0,com.facebook.katana,privacy at least put some option appear offli...,0
1,com.facebook.katana,"messenger issues ever since the last update, ...",0
2,com.facebook.katana,profile any time my wife or anybody has more ...,0
3,com.facebook.katana,the new features suck for those of us who don...,0
4,com.facebook.katana,forced reload on uploading pic on replying co...,0


Una vez se ha cargado correctamente la información en el Data Frame, es interesante evaluar **qué tipo y cuánta información se tiene**. Para ello, se recurre al atributo `.shape` del Data Frame:

In [3]:
# Se utiliza el atributo shape del Data Frame para conocer cuánta información está cargada:
print (f" El conjunto de datos cuenta con {df.shape[0]} comentarios con total de {df.shape[1]} tipos de información sobre ellos");

 El conjunto de datos cuenta con 891 comentarios con total de 3 tipos de información sobre ellos


## Paso 2 - Análisis Exploratorio de Datos:

Como se anticipaba en la lectura de las intrucciones del proyecto, el **conjunto de datos parte de tres variables**: 

- `package_name` : Nombre de la aplicación móvil (variable categórica)

- `review` : Comentario sobre la aplicación móvil (variable categórica)

- `polarity` : Variable objetivo que categoriza con un 0 un comentario positivo y con 1 un comentario positivo

Tal y como se indica en dichas instrucciones, de las **variables predictoras** (`package_name` y  `review`) solo **interesa** realmente el **comentario**, dado que para clasificar si un comentario es o no positivo dependerá de su contenido y no de en qué aplicación se ha escrito. 

Esto implica que hay que **eliminar** la columna `package_name`, para lo que se va a utilizar el método `.drop()`

In [4]:
# Se elimina la variable que no aporta información sobre el objetivo:
df = df.drop(['package_name'], axis = 1);

# Se comprueba que se ha eliminado correctamente:
print (f" El conjunto de datos cuenta con {df.shape[0]} comentarios con total de {df.shape[1]} tipos de información sobre ellos");

 El conjunto de datos cuenta con 891 comentarios con total de 2 tipos de información sobre ellos


En esta ocasión **no es necesario realizar un Análisis Exploratorio de Datos** (EDA) en tanto que la **variable predictora** con la que se cuenta es de **tipo texto**. En caso de tener un número más elevado de variables independientes y que fuesen de tipo numérico, sí sería necesario realizar este paso. 

## Paso 3 - Partición del Conjunto de Datos:

Como se acaba de mencionar, la **variable independiente es el contenido del comentario** `review` mientras que la **variable a predecir es si es o no positivo** dicho comentario `polarity`. 

Por esta razón, en primer lugar, se va a realizar esta **separación entre variable independiente y dependiente**:

In [5]:
# Se separa la variable dependiente: 
y = df.drop (['review'], axis = 1);

# Se separa la variable independiente (en este caso es un simple array al solo ser una variable):
x = df.drop (['polarity'], axis = 1);

Ahora que ya se han separado las variables se puede proceder a realizar la **partición del conjunto de datos**, teniendo una parte para **entrenar al modelo** (*train*) y otra para **probarlo** (*test*). De esta forma, se puede evaluar el rendimiento sin inferir en su **capacidad predictiva**.

In [6]:
# Se realiza la partición, explicitando el tamaño del test set:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42);
x_train = x_train['review'].tolist();
x_test = x_test['review'].tolist();

## Paso 4 - Vectorización del texto:

Los **algoritmos de Machine Learning no pueden trabajar directamente con datos en formato de texto**. Por esta razón, en general, se suele realizar una **codificación de variables categóricas a numéricas** como parte del EDA. Sin embargo, como se ha explicado antes, en este caso la única variable predictora con la que se cuenta es en formato texto por lo que un EDA carecía de sentido. 

El **objetivo principal de predicción** en este caso es saber si un c**omentario es o no positivo**. Es por ello por lo que se está trabajando con el **contenido de esta opinión**, lo que implica que el **interés recae en analizar las palabras** como tal. 

En este sentido, se va a utilizar una **instancia de la clase** `CountVectorizer` de forma que se pueda **convertir el texto en una matrix** donde **cada línea haga referencia a un comentario** y **cada columna a las palabras únicas** encontradas en el conjunto de entrenamiento. Así se consigue **información de la frecuencia de cada palabra** dentro de cada comentario, es decir, cada celda de la matriz muestra cuántas veces aparece esa palabra en el comentario correspondiente. 

Teniendo esto en cuenta, tras instanciar el vectorizador, se **entrenará** con el conjunto de *train* y se **aplicará** al de *test*. 

In [7]:
# Se declara una instancia del vectorizador: 
vectorizer = CountVectorizer();

# Se entrena y aplica la vectorización sobre el train set:
x_train_vec = vectorizer.fit_transform(x_train);

# Se aplica el vectorizador entrenador sobre el test set:
x_test_vec = vectorizer.transform (x_test);

In [8]:
print(f'Email de prueba: {x_test[1]}')
print('\nPalabras del conjunto de entrenamiento que también aparecen en el email de test junto con su aparición:\n')
for i, cont in enumerate(x_test_vec.toarray()[1]):
  if cont!=0:
    print(f'Palabra: "{vectorizer.get_feature_names_out()[i]}"')
    print(f'--> Aparece {cont} veces en el email.')

Email de prueba:  whatsapp i use this app now that blackberry messenger has basically gone away. my friends & family live all over the world. this really helps keep us in touch!

Palabras del conjunto de entrenamiento que también aparecen en el email de test junto con su aparición:

Palabra: "all"
--> Aparece 1 veces en el email.
Palabra: "app"
--> Aparece 1 veces en el email.
Palabra: "away"
--> Aparece 1 veces en el email.
Palabra: "basically"
--> Aparece 1 veces en el email.
Palabra: "family"
--> Aparece 1 veces en el email.
Palabra: "friends"
--> Aparece 1 veces en el email.
Palabra: "gone"
--> Aparece 1 veces en el email.
Palabra: "has"
--> Aparece 1 veces en el email.
Palabra: "helps"
--> Aparece 1 veces en el email.
Palabra: "in"
--> Aparece 1 veces en el email.
Palabra: "keep"
--> Aparece 1 veces en el email.
Palabra: "live"
--> Aparece 1 veces en el email.
Palabra: "messenger"
--> Aparece 1 veces en el email.
Palabra: "my"
--> Aparece 1 veces en el email.
Palabra: "now"
--> Ap

In [9]:
# Inicializar y entrenar el clasificador Naive Bayes Bernouilli
bernoulli_model = BernoulliNB().fit(x_train_vec, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred_bernouilli = bernoulli_model.predict(x_test_vec)

# Evaluar el rendimiento del modelo
print(classification_report(y_test, y_pred_bernouilli))

              precision    recall  f1-score   support

           0       0.86      0.94      0.90       126
           1       0.81      0.64      0.72        53

    accuracy                           0.85       179
   macro avg       0.84      0.79      0.81       179
weighted avg       0.85      0.85      0.84       179



/home/vscode/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [10]:
# Inicializar y entrenar el clasificador Naive Bayes Multinomial
multinomial_model = MultinomialNB().fit(x_train_vec, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred_multinomial = multinomial_model.predict(x_test_vec)

# Evaluar el rendimiento del modelo
print(classification_report(y_test, y_pred_multinomial))

              precision    recall  f1-score   support

           0       0.85      0.95      0.90       126
           1       0.84      0.58      0.69        53

    accuracy                           0.84       179
   macro avg       0.84      0.77      0.79       179
weighted avg       0.84      0.84      0.83       179



/home/vscode/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [11]:
# Inicializar y entrenar el clasificador Naive Bayes Gaussia
gaussian_model = GaussianNB().fit(x_train_vec.toarray(), y_train)

# Realizar predicciones en el conjunto de prueba
y_pred_gaussian = gaussian_model.predict(x_test_vec.toarray())

# Evaluar el rendimiento del modelo
print(classification_report(y_test, y_pred_gaussian))

              precision    recall  f1-score   support

           0       0.84      0.89      0.86       126
           1       0.70      0.60      0.65        53

    accuracy                           0.80       179
   macro avg       0.77      0.75      0.76       179
weighted avg       0.80      0.80      0.80       179



/home/vscode/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

# Definir distribuciones de parámetros aleatorios para RandomizedSearchCV
param_dist = {
    'n_estimators': randint(100, 500),  # Número de árboles aleatorios entre 100 y 500
    'max_depth': [None, 10, 20, 30, 50],  # Profundidad máxima de los árboles
    'min_samples_split': randint(2, 10),  # Mínimo número de muestras necesarias para dividir un nodo
    'min_samples_leaf': randint(1, 5),  # Mínimo número de muestras necesarias en una hoja
    'max_features': ['auto', 'sqrt', 'log2'],  # Métodos de selección de características
    'bootstrap': [True, False]  # Muestreo con o sin reemplazo
};

# Inicializar el clasificador Random Forest
rf_model = RandomForestClassifier(random_state=42);

# Configurar RandomizedSearchCV para buscar parámetros aleatorios
random_search = RandomizedSearchCV(
    estimator=rf_model, 
    param_distributions=param_dist, 
    n_iter=10,  # Número de combinaciones aleatorias a probar
    cv=5,  # Número de pliegues para la validación cruzada
    n_jobs=-1,  # Usar todos los núcleos disponibles
    verbose=1,  # Mostrar el progreso
    random_state=42
);

# Entrenar el modelo con RandomizedSearchCV
random_search.fit(x_train_vec, y_train);

# Obtener el mejor modelo entrenado con los parámetros óptimos
best_rf_model_random = random_search.best_estimator_;

# Evaluar el rendimiento del modelo con los mejores parámetros
y_pred_best_rf_random = best_rf_model_random.predict(x_test_vec);

# Mostrar el reporte de clasificación
print(classification_report(y_test, y_pred_best_rf_random));

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/home/vscode/.local/lib/python3.10/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/home/vscode/.local/lib/python3.10/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/home/vscode/.local/lib/python3.10/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/home/vscode/.local/lib/python3.10/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape o

              precision    recall  f1-score   support

           0       0.81      1.00      0.90       126
           1       1.00      0.45      0.62        53

    accuracy                           0.84       179
   macro avg       0.91      0.73      0.76       179
weighted avg       0.87      0.84      0.82       179



In [93]:
vectorizer.get_feature_names_out()

array(['000', '04', '0x', ..., 'žŕ', 'žŕľ', 'ˇŕ'],
      shape=(3553,), dtype=object)

In [94]:
# Convertimos la matriz dispersa a un DataFrame
df_vec = pd.DataFrame(x_train_vec.toarray(), columns=vectorizer.get_feature_names_out())

# Mostramos la matriz de forma clara
print(df_vec)

     000  04  0x  10  100  101  11  113mb  1186  12  125  13  14  14th  15  \
0      0   0   0   0    0    0   0      0     0   0    0   0   0     0   0   
1      0   0   0   0    0    0   0      0     0   0    0   0   0     0   0   
2      0   0   0   0    0    0   0      0     0   0    0   0   0     0   0   
3      0   0   0   0    0    0   0      0     0   0    0   0   0     0   0   
4      0   0   0   0    0    0   0      0     0   0    0   0   0     0   0   
..   ...  ..  ..  ..  ...  ...  ..    ...   ...  ..  ...  ..  ..   ...  ..   
707    0   0   0   0    0    0   0      0     0   0    0   0   0     0   0   
708    0   0   0   0    0    0   0      0     0   0    0   0   0     0   0   
709    0   0   0   0    0    0   0      0     0   0    0   0   0     0   0   
710    0   0   0   0    0    0   0      0     0   0    0   0   0     0   0   
711    0   0   0   0    0    0   1      0     0   0    0   0   0     0   0   

     15mb  16  17  180k  1990s  1lac  1st  20  200  2014  2015 